In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from simport make_scorer klearn.metrics 
from statistics import mean

In [ ]:
cd '/content/drive/MyDrive/AMIT_JOB/customer_chrun'

/content/drive/MyDrive/AMIT_JOB/customer_chrun


In [ ]:
#read data set 

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/AMIT_JOB/customer_chrun/features_data.csv')
labels = pd.read_csv('/content/drive/MyDrive/AMIT_JOB/customer_chrun/labels_data.csv')


In [ ]:
features.shape

(4250, 28)

In [ ]:
labels.shape

(4250, 1)

In [ ]:
features.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,area_code_408,area_code_415,area_code_510
0,0,0,0,0,1,0,0,0,0.512077,0,1,0.65,0.436770,0.721154,0.436805,0.484669,0.523364,0.484572,0.702408,0.532710,0.702341,0.748201,0.3,0.749333,0.333333,0.0,1.0,0.0
1,0,1,0,0,0,0,0,0,0.657005,0,0,0.00,0.717869,0.634615,0.718099,0.210196,0.588785,0.209909,0.357035,0.542056,0.357023,0.640288,0.5,0.640000,0.000000,0.0,1.0,0.0
2,0,0,0,0,1,0,0,0,0.400966,1,0,0.00,0.910309,0.221154,0.910617,0.503879,0.383178,0.503694,0.486080,0.401869,0.485786,0.237410,0.7,0.237333,0.666667,1.0,0.0,0.0
3,0,0,0,0,1,0,0,0,0.357488,1,0,0.00,0.454296,0.625000,0.454398,0.310307,0.700935,0.310300,0.448457,0.700935,0.448161,0.489209,0.3,0.490667,1.000000,0.0,1.0,0.0
4,0,0,0,0,1,0,0,0,0.579710,0,1,0.60,0.631271,0.384615,0.631345,0.503879,0.570093,0.503694,0.545147,0.672897,0.545151,0.302158,0.7,0.304000,1.000000,0.0,0.0,1.0


In [ ]:
labels.head()

,churn
0,0
1,0
2,0
3,0
4,0


In [ ]:
#split data into train,test 

In [ ]:
x_train, x_test , y_train, y_test = train_test_split(features, labels , test_size=0.3, random_state=0)

In [ ]:
x_train.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,area_code_408,area_code_415,area_code_510
3897,0,0,0,1,0,0,0,0,0.415459,0,1,0.700,0.578007,0.730769,0.578160,0.681936,0.476636,0.681877,0.440181,0.037383,0.439799,0.820144,0.6,0.821333,0.333333,0.0,1.0,0.0
1950,0,0,0,0,1,0,0,0,0.541063,0,0,0.000,0.724399,0.576923,0.724570,0.722571,0.869159,0.722295,0.513920,0.644860,0.513378,0.532374,0.2,0.533333,1.000000,0.0,0.0,1.0
2600,0,1,0,0,0,0,0,0,0.357488,0,0,0.000,0.929897,0.557692,0.930233,0.456963,0.635514,0.456758,0.630926,0.654206,0.630435,0.374101,0.5,0.376000,0.000000,0.0,1.0,0.0
264,0,0,1,0,0,0,0,0,0.570048,0,1,0.725,0.286254,0.173077,0.286350,0.711119,0.626168,0.710995,0.300602,0.280374,0.300167,0.309353,0.3,0.309333,1.000000,0.0,1.0,0.0
385,0,0,0,0,1,0,0,0,0.434783,0,0,0.000,0.340893,0.259615,0.340950,0.483931,0.373832,0.483703,0.800602,0.401869,0.800167,0.187050,0.2,0.186667,0.000000,1.0,0.0,0.0


In [ ]:
y_train.head()

,churn
3897,0
1950,1
2600,1
264,0
385,0


In [ ]:
#from EDA we find that the data is unbalanced so we will upsampling data 

In [ ]:
sm = SMOTE(sampling_strategy = 1, random_state=1)   
x_train_s, y_train_s = sm.fit_resample(x_train, y_train.values.ravel())
x_test_s, y_test_s = sm.fit_resample(x_test, y_test.values.ravel())
                                         

In [ ]:
print('After upsampling count of label 0 {}'.format(sum(y_train_s==0)))
print('After upsampling count of label 1 {}'.format(sum(y_train_s==1)))
print('After upsampling count of label 0 {}'.format(sum(y_test_s==0)))
print('After upsampling count of label 1 {}'.format(sum(y_test_s==1)))

After upsampling count of label 0 2550
After upsampling count of label 1 2550
After upsampling count of label 0 1102
After upsampling count of label 1 1102


In [ ]:
#compare between classifcation models to choose best one 
#1-random forest
#2-Support Vector classifier 
#3-Logistic Regression


In [ ]:
#we care the most for the false negative 
#we need to reduce the number of chrun clients that the model predict that the won't chrun 

In [ ]:
#create recall function 
def my_custom_loss_func(y_true, y_pred):
  return recall_score(y_true, y_pred, average='binary') 
  


In [ ]:
# evaluate a logistic regression model using repeated k-fold cross-validation
cv = 5

# create model
model = LogisticRegression()

my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
scores = cross_val_score(model, x_train_s, y_train_s,scoring=my_scorer,cv=cv, n_jobs=-1)

# report performance

print("Logistic Regression scores % s" % scores)
print("mean of scores % s" % (mean(scores)))
#print(mean(scores))
#recall
#[0.68101761 0.71428571 0.69726562 0.71289062 0.72994129]
model.fit(x_train_s, y_train_s)
y_pred = model.predict(x_test_s)

print('Logistic Regression recall score : ')
print('{}'.format(recall_score(y_test_s, y_pred, average='binary')))

Logistic Regression scores [0.69803922 0.69215686 0.72156863 0.70784314 0.72156863]
mean of scores 0.7082352941176471
Logistic Regression recall score : 
0.6569872958257713


In [ ]:
# evaluate a RandomForest model using  cross-validation
from sklearn import tree
cv = 5

# create model
rf_model = RandomForestClassifier()
# evaluate model
#scoring=recall_score(x_train_s, y_train_s, average=None)
my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
scores = cross_val_score(rf_model, x_train_s, y_train_s,scoring=my_scorer,cv=cv, n_jobs=-1)

# report performance
print(" Random Forest scores % s" % scores)
print("mean of scores % s" % (mean(scores)))
#print(mean(scores))
#recall
#[0.68101761 0.71428571 0.69726562 0.71289062 0.72994129]
rf_model.fit(x_train_s, y_train_s)
y_pred = rf_model.predict(x_test_s)
#recall_score(y_test_s, y_pred, average='binary')
print('Random Forest recall score : ')
print('{}'.format(recall_score(y_test_s, y_pred, average='binary')))


 Random Forest scores [0.80392157 0.93921569 0.95882353 0.93921569 0.95882353]
mean of scores 0.92
Random Forest recall score : 
0.6987295825771325


In [ ]:
# evaluate a svc model using  cross-validation

cv = 5

# create model
model = SVC(kernel='rbf', decision_function_shape='ovr')
# evaluate model
#scoring=recall_score(x_train_s, y_train_s, average=None)
my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
scores = cross_val_score(model, x_train_s, y_train_s,scoring=my_scorer,cv=cv, n_jobs=-1)
# report performance

print("svc scores % s" % scores)
print("mean of scores % s" % (mean(scores)))

model.fit(x_train_s, y_train_s)
y_pred = model.predict(x_test_s)
#recall_score(y_test_s, y_pred, average='binary')
print('svc recall score : ')
print('{}'.format(recall_score(y_test_s, y_pred, average='binary')))

svc scores [0.74901961 0.81764706 0.82156863 0.82156863 0.82745098]
mean of scores 0.8074509803921568
svc recall score : 
0.5970961887477314


In [ ]:
#we care about false negative and false positive 
#false negative means that the model predict that the client won't chrun but it chrun 
#so we care aboout recall 
#Here the highest value for recall find in random forest model 










In [ ]:
### Create a Pickle file using serialization 
import pickle


In [ ]:
pickle_out = open("random_forest_classifier.pkl","wb")
pickle.dump(rf_model, pickle_out)
pickle_out.close()

In [ ]:
#test model 
example= [[1.000000,0.000000, 0.000000,0.000000, 0.000000,0.000000,0.000000,0.000000, 
     0.869565,0.000000,1.000000, 0.400000,0.554983,0.355769,0.555106, 0.158109, 0.672897,0.157757,
     0.519940,0.598131, 0.520067,0.460432,0.100000, 0.461333,0.333333,  0.000000,0.000000, 1.000000]]

In [ ]:
pred=rf_model.predict(example)[0]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
print(pred)

0


In [ ]:
if pred==0:
  print('Will stay')
else:
  print('will chrun')  

Will stay


## Conclusion
Here I will give data to different machine learning models and then decide which is better 

But before that, the data  need to be splited into train, test 

from EDA we find that the data is unbalanced so we will upsampling data 
 
**compare between classifcation models to choose the best one**

1-random forest

2-Support Vector classifier 

3-Logistic Regression
 

 
 
 
 
 
 
